In [117]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend
from networkx.drawing.nx_agraph import from_agraph


# CLS assignment 2: Networked part

In [118]:
# set MPL to render plots by chosing backend
mpl.use('TkAgg')

In [119]:
# assuming an undirected graph

# parameters

N = 1000 # number of nodes
p = 0.1 # probability link creation
# Example: Creating a simple graph with 1000 nodes and a 0.1 probability of edge creation
g = nx.erdos_renyi_graph(N, p)



In [120]:
SIR_model_randomgraph = ep.SIRModel(g)

In [121]:
config = mc.Configuration()
config.add_model_parameter('beta', 0.002)  # Infection probability
config.add_model_parameter('gamma', 0.05)  # Recovery probability
SIR_model_randomgraph.set_initial_status(config)

# Example: Infecting node 0
config.add_model_initial_configuration("Infected", 0.05*N)




C:\Users\nitai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ndlib\models\DiffusionModel.py:120: UserWarning: Initial infection missing: a random sample of 5% of graph nodes will be set as infected
  warnings.warn('Initial infection missing: a random sample of 5% of graph nodes will be set as infected')


In [122]:
iterations = SIR_model_randomgraph.iteration_bunch(200)  # Execute 200 iterations
trends = SIR_model_randomgraph.build_trends(iterations)

In [123]:
# for iteration, status in enumerate(iterations):
#     print(f"Iteration {iteration}:")
#     print("Node status:", status['status'])
#     print("-----")

In [124]:
len(trends)

1

In [126]:
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend

# Visualize the diffusion trend
viz = DiffusionTrend(SIR_model_randomgraph, trends)
viz.plot()
plt.show()


Writing own simulation functions

In [127]:
def simulate_sir_on_network(N, k, network_type='erdos_renyi', p=None, beta=0.002, gamma=0.05, initial_i_proportion=0.1, iterations=200, plot=False, return_network = True):
    """
    Simulate the SIR (Susceptible, Infected, Recovered) model on a network and return the proportions of each compartment over time.
    
    Parameters:
    - N (int): Total number of nodes in the network.
    - k (int): Number of nodes each node is connected to.
    - network_type (str): Type of network to be created. Supported types: 'erdos_renyi', 'watts_strogatz', 'barabasi_albert'.
    - p (float, optional): Probability for edge creation (relevant for 'erdos_renyi' and 'watts_strogatz'). Default is None.
    - beta (float): Probability of transmission from an infected to a susceptible node during an interaction. Default is 0.002.
    - gamma (float): Probability of an infected node recovering in one iteration. Default is 0.05.
    - initial_i_proportion (float): Proportion of initially infected nodes. Default is 0.1.
    - iterations (int): Number of iterations for the simulation. Default is 200.
    - graph (bool): Whether to plot the proportions of S, I, R over time. Default is False.
    
    Returns:
    - list: A list of dictionaries with the proportions of 'S', 'I', and 'R' at each iteration.
    
    Raises:
    - ValueError: If an unsupported network type is provided or if the 'p' parameter is missing for certain network types.
    """

    # Initialize the network based on the specified network type
    if network_type == 'erdos_renyi':
        if p is None:
            raise ValueError("The 'p' parameter is required for the 'erdos_renyi' network type.")
        network = nx.erdos_renyi_graph(N, p)
    elif network_type == 'watts_strogatz':
        if p is None:
            raise ValueError("The 'p' parameter is required for the 'watts_strogatz' network type.")
        network = nx.watts_strogatz_graph(N, k, p)
    elif network_type == 'barabasi_albert':
        network = nx.barabasi_albert_graph(N, k)
    else:
        raise ValueError(f"Unsupported network type: {network_type}")

    # Set initial state for all nodes as Susceptible (S)
    states = {node: 'S' for node in network.nodes()}
    
    # Randomly choose nodes to set as initial Infected (I) state
    initial_infected_nodes = random.sample(list(network.nodes()), int(N * initial_i_proportion))
    for node in initial_infected_nodes:
        states[node] = 'I'
    
    # Track the proportions of S, I, R at each iteration
    trends = [{'S': (N - len(initial_infected_nodes))/N, 'I': len(initial_infected_nodes)/N, 'R': 0}]
    
    # Simulate the SIR model over the specified iterations
    for _ in range(iterations):
        new_states = states.copy()
        for node, state in states.items():
            # If node is infected, it may recover or infect its neighbors
            if state == 'I':
                # Node recovers with probability gamma
                if random.random() < gamma:
                    new_states[node] = 'R'
                    continue
                
                # For each neighbor, it can get infected with probability beta
                neighbors = list(network.neighbors(node))
                for neighbor in neighbors:
                    if new_states[neighbor] == 'S' and random.random() < beta:
                        new_states[neighbor] = 'I'
        
        # Update states for the next iteration
        states = new_states
        
        # Append the proportions of S, I, R to the trends list
        trends.append({'S': list(states.values()).count('S')/N, 
                       'I': list(states.values()).count('I')/N, 
                       'R': list(states.values()).count('R')/N})
    
    # Plot the trends if plot=True
    if plot:
        S = [t['S'] for t in trends]
        I = [t['I'] for t in trends]
        R = [t['R'] for t in trends]
        
        plt.figure(figsize=(10, 6))
        plt.plot(S, label='Susceptible', color='blue')
        plt.plot(I, label='Infected', color='red')
        plt.plot(R, label='Recovered', color='green')
        plt.xlabel('Iterations')
        plt.ylabel('Proportion of Population')
        plt.title('SIR Model on Network (Proportions)')
        plt.legend()
        plt.grid(True)
        plt.show()
    if return_network:
        return trends, network
    else:
        return trends


In [128]:
def multi_simulate_sir_on_network(N, k, network_type='erdos_renyi', p=None, beta=0.002, gamma=0.05, initial_i_proportion=0.1, iterations=200, plot=False):
    """
    Simulate the SIR model on a network for multiple parameter settings and return the results.
    
    Parameters:
    - N (int or list): Total number of nodes in the network or list of such values.
    - k (int or list): Number of nodes each node is connected to or list of such values.
    - network_type (str): Type of network to be created. Supported types: 'erdos_renyi', 'watts_strogatz', 'barabasi_albert'.
    - p (float or list, optional): Probability for edge creation (relevant for 'erdos_renyi' and 'watts_strogatz') or list of such values. Default is None.
    - beta (float or list): Probability of transmission from an infected to a susceptible node during an interaction or list of such values. Default is 0.002.
    - gamma (float or list): Probability of an infected node recovering in one iteration or list of such values. Default is 0.05.
    - initial_i_proportion (float or list): Proportion of initially infected nodes or list of such values. Default is 0.1.
    - iterations (int): Number of iterations for the simulation. Default is 200.
    - graph (bool): Whether to plot the proportions of S, I, R over time for each simulation. Default is False.
    
    Returns:
    - list: A list of dictionaries containing the simulation parameters and the trends of 'S', 'I', and 'R' for each parameter combination.
    """

    # Ensure parameters are in list format for iteration
    if not isinstance(N, list): N = [N]
    if not isinstance(k, list): k = [k]
    if not isinstance(p, list): p = [p]
    if not isinstance(beta, list): beta = [beta]
    if not isinstance(gamma, list): gamma = [gamma]
    if not isinstance(initial_i_proportion, list): initial_i_proportion = [initial_i_proportion]
    
    results = []
    
    # Iterate over all combinations of parameter values
    for n in N:
        for degree in k:
            for probability in p:
                for b in beta:
                    for g in gamma:
                        for init_inf in initial_i_proportion:
                            
                            # Simulate the SIR model with the current parameter combination
                            trends = simulate_sir_on_network(n, degree, network_type, probability, b, g, init_inf, iterations, plot)
                            
                            # Store the parameters and their corresponding trends
                            params = {
                                "N": n,
                                "k": degree,
                                "network_type": network_type,
                                "p": probability,
                                "beta": b,
                                "gamma": g,
                                "initial_i_proportion": init_inf,
                                "iterations": iterations
                            }
                            results.append({"parameters": params, "trends": trends})
                            
    return results


Ideas for parameter sets:

1. Random Networks:

Generate random networks with varying probabilities of edge creation (p) to explore the impact of network density on disease spread.

Parameter Set 1: Low Density

N (Number of Nodes): 500
k (Average Degree): 5
p (Probability of Edge Creation): 0.05
Parameter Set 2: Medium Density

N (Number of Nodes): 500
k (Average Degree): 5
p (Probability of Edge Creation): 0.2
Parameter Set 3: High Density

N (Number of Nodes): 500
k (Average Degree): 5
p (Probability of Edge Creation): 0.5
2. Small-World Networks:

Generate small-world networks with varying rewiring probabilities (p) to investigate the impact of network clustering on disease spread.

Parameter Set 4: Low Clustering (High Rewiring)

N (Number of Nodes): 500
k (Average Degree): 5
p (Rewiring Probability): 0.8
Parameter Set 5: Moderate Clustering (Medium Rewiring)

N (Number of Nodes): 500
k (Average Degree): 5
p (Rewiring Probability): 0.4
Parameter Set 6: High Clustering (Low Rewiring)

N (Number of Nodes): 500
k (Average Degree): 5
p (Rewiring Probability): 0.1
3. Scale-Free Networks (Barabási-Albert Model):

Generate scale-free networks using the Barabási-Albert model with varying initial attachment values (m0) to explore the impact of hub formation on disease spread.

Parameter Set 7: Low Hub Formation

N (Number of Nodes): 1000
m0 (Initial Attachment): 2
Parameter Set 8: Moderate Hub Formation

N (Number of Nodes): 1000
m0 (Initial Attachment): 5
Parameter Set 9: High Hub Formation

N (Number of Nodes): 1000
m0 (Initial Attachment): 10


In [129]:
# Parameter sets for multi_simulate_sir_on_network function

# Random Networks
param_set_random_low_density = {
    "N": [500],
    "k": [5],
    "network_type": "erdos_renyi",
    "p": [0.05, 0.2, 0.5]  # Varying values for 'p'
}

# Small-World Networks
param_set_small_world_low_clustering = {
    "N": [500],
    "k": [5],
    "network_type": "watts_strogatz",
    "p": [0.8, 0.4, 0.1]  # Varying values for 'rewiring_prob'
}

# Scale-Free Networks (Barabási-Albert Model)
param_set_scale_free_low_hub_formation = {
    "N": [1000],
    "k": [2, 5, 10],  # Varying values for 'k'
    "network_type": "barabasi_albert"
}


In [132]:
# Call the function for each parameter set and store the results
results_random_low_density = multi_simulate_sir_on_network(**param_set_random_low_density, plot=True)
results_small_world_low_clustering = multi_simulate_sir_on_network(**param_set_small_world_low_clustering, plot=True)
results_scale_free_low_hub_formation = multi_simulate_sir_on_network(**param_set_scale_free_low_hub_formation, plot=True)


In [135]:
# network_types = ['erdos_renyi', 'watts_strogatz', 'barabasi_albert']
# params = [0.1, 0.2, 0.3, 0.4, 0.5]

In [136]:
# beta_list = [0.002, 0.004, 0.01, 0.03, 0.05]
# gamma_list = [0.05, 0.1, 0.15, 0.2, 0.25]
# reverse_gamma_list = gamma_list[::-1]

In [137]:
def extended_combined_network_statistics(graph, plot=False, caption=None):
    """
    Compute basic statistics, centrality measures (including min and max values), and optional visualizations 
    for a given network/graph, with checks for connectivity.
    
    Parameters:
    - graph (networkx.Graph): The input graph/network for which statistics and visualizations are to be computed.
    - plot (bool): Whether to plot the histograms for node degrees and centrality measures. Default is False.
    - caption (str): Caption to display below the plots if plot=True. Default is None.
    
    Returns:
    - stats_df (pandas.DataFrame): A DataFrame containing the computed statistics of the graph.
    - degree_dist_df (pandas.DataFrame): A DataFrame containing the degree distribution of the nodes in the graph.
    """
    
    # Basic statistics
    stats = {
        "Average Degree": sum(dict(graph.degree()).values()) / len(graph),
        "Clustering Coefficient": nx.average_clustering(graph)
    }
    
    # Check for connectivity
    if nx.is_connected(graph):
        stats["Average Shortest Path Length"] = nx.average_shortest_path_length(graph)
        stats["Max Path Length"] = nx.diameter(graph)
    else:
        stats["Average Shortest Path Length"] = None
        stats["Max Path Length"] = None
    
    # Centrality measures
    degree_centrality = nx.degree_centrality(graph)
    eigenvector_centrality = nx.eigenvector_centrality(graph, max_iter=500)
    betweenness_centrality = nx.betweenness_centrality(graph)
    
    # Adding average, min, and max values of the centrality measures
    stats["Average Degree Centrality"] = sum(degree_centrality.values()) / len(graph)
    stats["Min Degree Centrality"] = min(degree_centrality.values())
    stats["Max Degree Centrality"] = max(degree_centrality.values())
    
    stats["Average Eigenvector Centrality"] = sum(eigenvector_centrality.values()) / len(graph)
    stats["Min Eigenvector Centrality"] = min(eigenvector_centrality.values())
    stats["Max Eigenvector Centrality"] = max(eigenvector_centrality.values())
    
    stats["Average Betweenness Centrality"] = sum(betweenness_centrality.values()) / len(graph)
    stats["Min Betweenness Centrality"] = min(betweenness_centrality.values())
    stats["Max Betweenness Centrality"] = max(betweenness_centrality.values())
    
    # Degree Distribution
    degree_sequence = sorted([d for n, d in graph.degree()], reverse=True)
    
    # Plotting histograms
    if plot:
        plt.figure(figsize=(20, 5))
        
        # Degree Distribution
        plt.subplot(1, 4, 1)
        plt.hist(degree_sequence, bins=50)
        plt.title("Degree Distribution")
        plt.xlabel("Degree")
        plt.ylabel("Number of Nodes")
        
        # Degree Centrality
        plt.subplot(1, 4, 2)
        plt.hist(list(degree_centrality.values()), bins=50, color='skyblue')
        plt.title("Degree Centrality")
        plt.xlabel("Degree Centrality")
        plt.ylabel("Number of Nodes")
    
        # Eigenvector Centrality
        plt.subplot(1, 4, 3)
        plt.hist(list(eigenvector_centrality.values()), bins=50, color='salmon')
        plt.title("Eigenvector Centrality")
        plt.xlabel("Eigenvector Centrality")
        plt.ylabel("Number of Nodes")
    
        # Betweenness Centrality
        plt.subplot(1, 4, 4)
        plt.hist(list(betweenness_centrality.values()), bins=50, color='lightgreen')
        plt.title("Betweenness Centrality")
        plt.xlabel("Betweenness Centrality")
        plt.ylabel("Number of Nodes")
        
        # Display caption below the plots if provided
        if caption:
            plt.figtext(0.5, -0.05, caption, ha="center", fontsize=12, bbox={"facecolor":"white", "alpha":0.5, "pad":5})
        
        plt.tight_layout()
        plt.show()
    
    stats_df = pd.DataFrame([stats])
    degree_dist_df = pd.DataFrame({"Degree": degree_sequence})
    
    return stats_df, degree_dist_df